### TODO: 

**Fix class imbalance** 
  * using NLP data augmentation https://neptune.ai/blog/data-augmentation-nlp So for examples containing rare genres and no common genres we can oversample with augmentation techniques. Problem: We have multi-label problem (many genres possible). To decide how to oversample we can have a "repeat ceiling". Better way would be to utlize 
  https://medium.com/thecyphy/handling-data-imbalance-in-multi-label-classification-mlsmote-531155416b87
  https://link.springer.com/chapter/10.1007/978-3-642-40846-5_16 to decide which examples to oversample.
  
  
**Fine-tune using distilgpt2** 
  * GPT2 is too large for our GPU so we use distilled version https://huggingface.co/distilgpt2
  * For best performance: Make LM dataset using plots only and finetune the model a bit on this. (Large text document with each plot after the other. See example notebook from hugging face)
  * Once the model outputs plot-like text, we want to train using the labeled dataset with genres and title.
  
** to start **
Since we won't have time to do everything probably, let's start with the current dataset and just try to see what happens if we simply finetune w/o any augmentation and no pre-training on plot text.
   


# About

This notebook implements a pre-trained GPT language model to generate text.

In [ ]:
!pip install transformers datasets git-lfs ipywidgets

In [ ]:
!conda list

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

# START


In [1]:
import math
import re
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

import torch
from datasets import load_dataset
from transformers import (
    TrainerCallback,
    GPT2Config,
    GPT2Tokenizer,
    GPT2LMHeadModel,
    AutoConfig,
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    AdamW,
    TrainingArguments,
    Trainer,
)

In [2]:
start_from_checkpoint = False

# Load pretrained tokenizer and model
finetuned_model_name = 'movie-plot-generator'

if start_from_checkpoint:
    config=AutoConfig.from_pretrained(finetuned_model_name)
    tokenizer = AutoTokenizer.from_pretrained(finetuned_model_name)
    model = AutoModelForCausalLM.from_pretrained(finetuned_model_name, config=config)
else:
    model_name = 'distilgpt2' 
    config=AutoConfig.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name, config=config)
    model = AutoModelForCausalLM.from_pretrained(model_name)

In [3]:
# Check of pre-trained model 
if start_from_checkpoint:
    generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
    stories = generator("<BOS> <action> Shrek in the swamp <SEP> He was in the ", max_length=200, num_return_sequences=2)
    print(*[story['generated_text'] + "\n\n\n------------------------\n" for story in stories])

# Load dataset

First, we load the dataset and split into train and validation 

In [4]:
# Load dataset from text file called "data.txt" and split into train/val
datasets = load_dataset("text", data_files="data_top_15_genres.txt")['train']
datasets = datasets.train_test_split(train_size=0.985, seed=42)
datasets['validation'] = datasets.pop('test')

# Removal of unwanted text parts if necessary 
def processText(example):
    #example['text'] = [ re.sub('<ref?(.*?)}}', '', text) for text in example['text'] ]
    #example['text'] = [ re.sub('<ref name?(.*?)<EOS>', ' <EOS>', text) for text in example['text'] ]
    #example['text'] = [ re.sub(' <!--', '', text) for text in example['text'] ]
    return example

datasets = datasets.map(processText, batched=True)
datasets

Using custom data configuration default-2fcf8d2135508f85



Dataset text downloaded and prepared to C:\Users\Anton\.cache\huggingface\datasets\text\default-2fcf8d2135508f85\0.0.0\e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5. Subsequent calls will reuse this data.


DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 36475
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 556
    })
})

As can be seen, the examples are of different lengths. Examples longer than 1024 tokens needs to be truncated as this is the maximum input to GPT2.

In [8]:
# Example
print(datasets['train'][0]['text'] + '\n')
print(datasets['train'][1]['text'] + '\n')
print(datasets['train'][3]['text'] + '\n')


<BOS> <action> Gundaraj <SEP> Ajay Chauhan lives with his parents and younger sister. He is in love with Pooja, and hopes to marry her someday. His father wants him to get a job and settle down, and then get married. Ajay applies for a job in Bombay, and soon receives a letter asking him to appear for an interview. He attends the interview, and is hired. Delighted to see all his dreams coming true, he goes to offer his thanks to God, and it is there a woman named Pratika Jetley sees him and notifies the police that he is indeed the one who had brutally raped three young women in a college campus. Ajay vehemently denies this, but is personally identified and criminally held responsible, convicted and sentenced to prison. Several years later he is released from prison, and finds out that his father and Pooja had committed suicide while his mother and sister are untraceable. He sets out to put his life together and meets with a ruthless police inspector, whose daughter was one of the rape

## Tokenization

We now need to tokenize the dataset. The original tokenizer don't have all special tokens we require.

In [9]:
print(*tokenizer.all_special_tokens)

<|endoftext|>


We need to add the special tokens that we use in our dataset. 

In [10]:
# Set special tokens
tokenizer.bos_token = '<BOS>'
tokenizer.eos_token = '<EOS>'
tokenizer.pad_token = '<PAD>'
tokenizer.sep_token = '<SEP>'

# Add special tokens for each genre
genres = ['romantic drama', 'short film', 'family film',
          'adventure', 'action/adventure', 'indie',
          'black-and-white', 'horror', 'crime fiction',
          'world cinema', 'action', 'thriller', 
          'romance film', 'comedy', 'drama']

print(f'Number of added genres: {len(genres)}')
new_special_tokens = ['<BOS>', '<EOS>', '<PAD>', '<SEP>']
new_special_tokens.extend([f'<{genre}>' for genre in genres])
special_tokens = tokenizer.additional_special_tokens

special_tokens.extend(new_special_tokens) 
new_special_tokens_dict = {'additional_special_tokens': special_tokens}
num_added_toks = tokenizer.add_special_tokens(new_special_tokens_dict)

# We must resize token embeddings since new special tokens were added
model.resize_token_embeddings(len(tokenizer))

print(model.config.vocab_size, tokenizer.vocab_size + len(tokenizer.get_added_vocab()))
assert(model.config.vocab_size == tokenizer.vocab_size + len(tokenizer.get_added_vocab()))
print(*tokenizer.all_special_tokens)

Number of added genres: 15
50276 50276
<BOS> <EOS> <|endoftext|> <SEP> <PAD> <romantic drama> <short film> <family film> <adventure> <action/adventure> <indie> <black-and-white> <horror> <crime fiction> <world cinema> <action> <thriller> <romance film> <comedy> <drama>


In [11]:
tokenizer.tokenize('<BOS> <drama> He was' )

['<BOS>', 'Ġ', '<drama>', 'ĠHe', 'Ġwas']

**Tokenize the dataset**

We tokenize the dataset. The tokenized examples contain the column names 'attention_mask' which is a mask for padding tokens and 'input_ids' which is the id of each token corrsponding to a word. We drop the text as that is not needed anymore. 

In [12]:
def tokenize_function(examples):
    """
    padding='max_length' to pad to a length specified by the max_length argument 
    or the maximum length accepted by the model.
    truncation=True to truncate each sequence to the maximum length accepted by the model
    """
    result = tokenizer(examples["text"], padding='max_length', truncation=True) # Max input according to model(1024)
    #result = tokenizer(examples["text"], max_length=512, padding='max_length', truncation=True)

    result["labels"] = result["input_ids"].copy()
    return result

tokenized_datasets = datasets.map(tokenize_function, batched=True, remove_columns=["text"])

Note that we duplicate the inputs to add our labels. This is because the model of the 🤗 Transformers library apply the shifting to the right, so we don't need to do it manually.

In [13]:
#Make dataset format pytorch tensors
tokenized_datasets.set_format("torch")
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'labels'],
        num_rows: 36475
    })
    validation: Dataset({
        features: ['attention_mask', 'input_ids', 'labels'],
        num_rows: 556
    })
})

In [14]:
# Finally, extract the datasets and select a subset if wanted
train_set = tokenized_datasets['train']#.select(list(range(10)))
valid_set = tokenized_datasets['validation']#.select(list(range(2)))

In [15]:
print(train_set, valid_set)

Dataset({
    features: ['attention_mask', 'input_ids', 'labels'],
    num_rows: 36475
}) Dataset({
    features: ['attention_mask', 'input_ids', 'labels'],
    num_rows: 556
})


### Training
First, setup training args.
The last argument to setup everything so we can push the model to the Hub regularly during training..

Then pass training args to Trainer.

In [16]:
class SaveTokenizer(TrainerCallback):
    """
    A callback used to save the tokenizer whenever a model checkpoint is saved.
    """
    def on_save(self, args, state, control, **kwargs):
        tokenizer.save_pretrained(f"./{finetuned_model_name}/")

        
ce_loss = torch.nn.CrossEntropyLoss()
        
def compute_metrics(eval_pred):
    """
    The compute function needs to receive a tuple (with logits and labels)
    and has to return a dictionary with string keys (the name of the metric) and float values.
    It will be called at the end of each evaluation phase on the whole arrays of predictions/labels.
    """
    logits, labels = eval_pred
    # Calculate perplexity https://huggingface.co/transformers/perplexity.html
    # "the exponentiation of the cross-entropy between the data and model predictions."
    
    perplexity = math.exp(ce_loss(logits, labels))
    
    return {'perplexity': perplexity}

In [17]:
torch.cuda.empty_cache()
batch_size = 2 # 1:34:39 for one epoch (no evaluation steps) with batch_size = 2

training_args = TrainingArguments(
    finetuned_model_name,
    evaluation_strategy = "no",
    num_train_epochs=1,
    learning_rate=1e-6,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=1,
    save_steps=2500,
    save_total_limit=1,
    push_to_hub=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=valid_set,
    compute_metrics=compute_metrics,
    callbacks=[SaveTokenizer],
)

In [ ]:
train_results=trainer.train()
pickle.dump(train_results, open("train_results.pickle", "wb")) #Load: train_results = pickle.load(open("train_results.pickle", "rb"))

model.save_pretrained(f"./{finetuned_model_name}/")
tokenizer.save_pretrained(f"./{finetuned_model_name}/")

***** Running training *****
  Num examples = 36475
  Num Epochs = 1
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 18238


Step,Training Loss
500,10.392100
1000,2.018200
1500,1.612000


In [ ]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

In [ ]:
tokenizer.push_to_hub(finetuned_model_name)
trainer.push_to_hub(finetuned_model_name)

In [ ]:
# Inference test
generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)
stories = generator("<BOS> <horror> Thovie <SEP>", max_length=1024, num_return_sequences=4)
print(*[story['generated_text'] + "\n\n\n------------------------\n" for story in stories])

### Push to HUB

Push tokenizer and model to hub

------
------
### Casual language modeling ## 
For causal language modeling (CLM) we are going to take all the texts in our dataset and concatenate them after they are tokenized. Then we will split them in examples of a certain sequence length. This way the model will receive chunks of contiguous text that may look like:
    
    part of text 1
    
or

    end of text 1 <BOS_TOKEN> beginning of text 2
    
 
depending on whether they span over several of the original texts in the dataset or not.
**Also the labels will be the same as the inputs, shifted to the left.**

Now for the harder part: we need to concatenate all our texts together then split the result in small chunks of a certain block_size. To do this, we will use the map method again, with the option batched=True. This option actually lets us change the number of examples in the datasets by returning a different number of examples than we got. This way, we can create our new samples from a batch of examples.
First, we grab the maximum length our model was pretrained with. This might be a big too big to fit in our GPU RAM, in that case decrease the size.

In [ ]:
#block_size = tokenizer.model_max_length
block_size = 128

Then we write the preprocessing function that will group our texts:


In [ ]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

First note that we duplicate the inputs for our labels. This is because the model of the 🤗 Transformers library apply the shifting to the right, so we don't need to do it manually.

Also note that by default, the map method will send a batch of 1,000 examples to be treated by the preprocessing function. So here, we will drop the remainder to make the concatenated tokenized texts a multiple of block_size every 1,000 examples. You can adjust this behavior by passing a higher batch size (which will also be processed slower). You can also speed-up the preprocessing by using multiprocessing:

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)
print(lm_datasets)

And we can check our datasets have changed: now the samples contain chunks of block_size contiguous tokens, potentially spanning over several of our original texts.

In [ ]:
print(tokenizer.decode(lm_datasets["train"][0]["input_ids"]))
print()
print(tokenizer.decode(lm_datasets["train"][1]["input_ids"]))

Now that the data has been cleaned, we're ready to instantiate our Trainer.